In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 13:37:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pandas as pd

In [9]:
df_pandas = pd.read_csv("head.csv")
df_pandas.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [11]:
from pyspark.sql import types
schema = types.StructType([
	types.StructField('dispatching_base_num', types.StringType(), True),
	types.StructField('pickup_datetime', types.TimestampType(), True),
	types.StructField('dropoff_datetime', types.TimestampType(), True),
	types.StructField('PULocationID', types.IntegerType(), True),
	types.StructField('DOLocationID', types.IntegerType(), True),
	types.StructField('SR_Flag', types.StringType(), True),
	types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [12]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv("fhvhv_tripdata_2021-06.csv")
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [14]:
df.head(5)

[Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 2, 41), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 7, 46), PULocationID=174, DOLocationID=18, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 16, 16), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 21, 14), PULocationID=32, DOLocationID=254, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 27, 1), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 42, 11), PULocationID=240, DOLocationID=127, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 6, 1, 0, 46, 8), dropoff_datetime=datetime.datetime(2021, 6, 1, 0, 53, 45), PULocationID=127, DOLocationID=235, SR_Flag='N', Affiliated_base_number='B02764'),
 Row(dispatching_base_num='B02510', pickup_datetime=datetime.d

In [15]:
df.repartition(12)
df.write.parquet('fhvhv_tripdata/2021/06/')

23/03/09 13:54:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [16]:
df = spark.read.parquet('fhvhv_tripdata/2021/06/')

In [18]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [26]:
from pyspark.sql import functions as F

### Question 3: How many taxi trips were started on June 15th?

In [30]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .select('pickup_date') \
    .filter(F.col('pickup_date') == '2021-06-15') \
    .count()

452470

### Question 4: How long is the longest trip in the dataset? 

In [33]:
df \
    .withColumn('trip_duration', df.dropoff_datetime - df.pickup_datetime) \
    .agg({'trip_duration': 'max'}) \
    .collect()

[Row(max(trip_duration)=datetime.timedelta(days=2, seconds=67964))]

### Question 6: What is the name of the most frequent pickup location zone?

In [41]:
schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True),
])
zone_df = spark.read.option("header", "true").schema(schema).csv('taxi+_zone_lookup.csv')
zone_df.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [48]:
df \
    .join(zone_df, df.PULocationID == zone_df.LocationID) \
    .groupBy("Zone") \
    .count() \
    .orderBy(F.col("count").desc()) \
    .take(1)

[Row(Zone='Crown Heights North', count=231279)]